# Psi4 Tutorial 03: Basis sets

In this tutorial you will learn more about basis sets

In [1]:
import psi4

## Basis set information and molecular orbital coefficients

Let's run a Hartree–Fock computation on H$_2$ using the def2-SVP basis set. This basis is defined as:
```
H     0
S   3   1.00
     13.0107010       0.19682158E-01  |
      1.9622572       0.13796524      | three s-type Gaussians combined to form one s orbital
      0.44453796      0.47831935      | 

S   1   1.00
      0.12194962      1.0000000       | one s-type Gaussian for a second s orbital

P   1   1.00
      0.8000000       1.0000000       | one set of 3 p-type Gaussians for a set of 3 p orbitals
      
          ^               ^
       exponent      coefficient
```      

Here we add the option `PRINT_MOS` via
```python
psi4.set_options({'PRINT_MOS' : True})
```
When `PRINT_MOS` is set to `True`, psi4 will print the molecular orbital coefficient matrix ($C_{\mu i}$) that tell us how the basis functions [$\chi_\mu(\mathbf{r})$] are combined to form molecular orbitals [$\phi(\mathbf{r})_i$]:
$$
\phi(\mathbf{r})_i = \sum_\mu^{\rm basis} \chi_\mu(\mathbf{r}) C_{\mu i}
$$

In [2]:
psi4.set_output_file('output.txt',False)

psi4.geometry("""
H
H 1 0.75
symmetry c1
""")

# print the MO coefficients
psi4.set_options({'PRINT_MOS' : True})

e, wfn = psi4.energy('SCF/def2-SVP',return_wfn=True)

If we examine the output file you will find a block that described the basis:
```
  ==> Primary Basis <==

  Basis Set: DEF2-SVP
    Blend: DEF2-SVP
    Number of shells: 6
    Number of basis functions: 10
    Number of Cartesian functions: 10
    Spherical Harmonics?: true
    Max angular momentum: 1
```
This tells us that there are 10 basis function (`Number of basis functions: 10`) and that we are using up to p functions (`Max angular momentum: 1`).

At the end of the computation we find the coefficient matrix $C_{\mu i}$:
```
 ==> Molecular Orbitals <==

                            1            2            3            4            5

 1    H1 s0         0.4034877    0.1449384    0.7040924    0.8728821    0.0000000
 2    H1 s0         0.1756845    1.9219361   -0.7270797   -1.5827242   -0.0000000
 3    H1 p0         0.0214187    0.0042600    0.0117829   -0.3296096   -0.0000000
 4    H1 p+1       -0.0000000   -0.0000000    0.0000000   -0.0000000    0.1239586
 5    H1 p-1       -0.0000000    0.0000000   -0.0000000   -0.0000000    0.5744515
 6    H2 s0         0.4034877   -0.1449384    0.7040924   -0.8728821    0.0000000
 7    H2 s0         0.1756845   -1.9219361   -0.7270797    1.5827242    0.0000000
 8    H2 p0        -0.0214187    0.0042600   -0.0117829   -0.3296096   -0.0000000
 9    H2 p+1       -0.0000000    0.0000000    0.0000000   -0.0000000    0.1239586
 10   H2 p-1       -0.0000000   -0.0000000    0.0000000   -0.0000000    0.5744515

            Ene    -0.5894693    0.1957203    0.4820146    0.9546651    1.4209743
            Sym             A            A            A            A            A
            Occ             2            0            0            0            0
```

On the left, you will find the list of atomic orbitals contained in the def2-SVP basis (`H1 s0`, `H1 s0`, `H1 p0`, etc.). Here `s0` stands for an s function, `p0,p+1,p-1` are **real combinations of spherical harmonic functions** and correspond to the `pz,px,py` orbitals (if you are curious about this point, check out the *Real orbitals* section of the wikipedia article https://en.wikipedia.org/wiki/Atomic_orbital).
Since this basis contains two s function, there are four lines that correspond to s atomic orbitals on the two hydrogen atoms.
Each column of numbers shows how these atomic orbitals are combined together to form a molecular orbital. For example, the first orbital is a combination of mostly s orbitals on the two hydrogens of the form
$$
\phi_1(\mathbf{r}) = 0.4034877 \mu_{\rm  H1 s0}(\mathbf{r}) + 0.4034877 \mu_{\rm  H2 s0}(\mathbf{r}) + 0.1756845 \mu_{\rm  H1 s0}(\mathbf{r}) + 0.1756845 \mu_{\rm  H2 s0}(\mathbf{r}) + \ldots
$$

Information about the basis set can be retrieved from the wavefunction object returned by psi4. Here we call the function `nbf()`, which returns the number of basis functions

In [3]:
basis = wfn.basisset()
nbf = basis.nbf()
print(nbf)

10


We can also visualize all the basis functions and orbitals. To do so we need to first generate grid representations of these quantities and then call `cubeprop`

In [4]:
psi4.set_options({'CUBEPROP_TASKS' : ['BASIS_FUNCTIONS','ORBITALS']})
psi4.cubeprop(wfn)

Next, we can use `fortecubeview` to visualize them

In [5]:
import fortecubeview
fortecubeview.plot()

CubeViewer: listing cube files from the directory .
Reading 30 cube files


/Users/fevange/miniforge3/envs/forte/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('./Phi_1.cube', './Phi_10.cube', './Phi_2.cub…

---

## Basis set extrapolation

Psi allows us to easily compute the Hartree–Fock energy extrapolated to the complete basis set (CBS) limit. Here we compute the basis set limit of the neon atom.
To request basis set extrapolation using the cc-pVTZ and cc-pVQZ basis set, we can call the energy function with the following input
```python
psi4.energy('SCF/cc-pV[TQ]Z')
```
Here `cc-pV[TQ]Z` means do an extrapolation using the cc-pVTZ and cc-pVQZ basis functions. This procedure computes the energy twice using the cc-pVTZ and cc-pVQZ basis sets and returns the extrapolated energy:

In [ ]:
psi4.geometry("""
Ne
""")

e_tq = psi4.energy('SCF/cc-pV[TQ]Z')
print(f'CBS estimate (TQ)  : {e_tq}')

The extrapolated energy (-128.546295) is lower than the energy computed in the next cell using the cc-pVQZ basis (-128.543466)

In [ ]:
psi4.geometry("""
Ne
""")

e_q = psi4.energy('SCF/cc-pVQZ')
print(f'CBS estimate (TQ)  : {e_tq}')
print(f'cc-pVQZ basis      : {e_q}')

To get a value close to the extrapolated result we need to use a cc-pV5Z basis. Since this increases the cost of a computation, it is a less effective way to improve the accuracy of our computation

In [ ]:
psi4.geometry("""
Ne
""")

e_5 = psi4.energy('SCF/cc-pV5Z')
print(f'CBS estimate (TQ)  : {e_tq}')
print(f'cc-pVQZ basis      : {e_q}')
print(f'cc-pV5Z basis      : {e_5}')